# Lab: Advanced Mission Planning — Adding Logical Constraints 🧠

Welcome, Mission Planner! This lab focuses on an advanced mission planning scenario. While a basic optimization might focus only on physical limits (mass, volume, etc.), real-world missions are governed by complex **operational rules** and **interdependencies**.

> *“If you take the high-gain antenna, you **must** also take the auxiliary power unit.”*
> *“You can pack either the drilling rig **or** the surface scanner, but there isn't room for both.”*

These aren't simple capacity limits; they are **logical constraints**. In this lab, you'll learn how to inject this critical mission logic directly into your quantum optimization model.

### Learning Goals 🎯

* Translate real-world operational rules into mathematical inequalities.
* Add multiple types of logical constraints to a `QuadraticProgram`.
* Analyze how adding constraints changes the optimal solution.
* Understand the trade-off between optimality and operational feasibility.

---
## 1. The Scenario: Mission Directives

You are planning the cargo manifest for a resupply mission to Mars. Your goal is to maximize the scientific value of the items you send, subject to the lander's physical limits and some new operational rules from Mission Control.

### 🚀 The Cargo Manifest

| Item # | Item Name          | Scientific Value | Mass (kg) | Volume (m³) | Power (kW) |
|--------|--------------------|------------------|-----------|-------------|------------|
| 0      | Soil Analyzer      | 12               | 5         | 2           | 4          |
| 1      | Drone Scout        | 10               | 3         | 4           | 5          |
| 2      | BioLab Kit         | 15               | 6         | 5           | 2          |
| 3      | Comm Booster       | 8                | 3         | 1           | 6          |
| 4      | Emergency Battery  | 5                | 2         | 3           | 1          |

**Physical Capacities:** Mass ≤ 12 kg, Volume ≤ 10 m³, Power ≤ 11 kW.




In [ ]:
# Step 1: Import all necessary libraries
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram

**Fill this code from your previous exercise solution**

In [ ]:

# 1. Define the problem variables for the new instance
values = np.array([ ?, ?, ?, ?, ? ])  # TODO: Add the new values array
weights = np.array([
    [ ?, ?, ?, ?, ? ],  # ← Constraint 0
    [ ?, ?, ?, ?, ? ],  # ← Constraint 1
    [ ?, ?, ?, ?, ? ]   # ← Constraint 2
])
capacities = np.array([ ?, ?, ? ])   # TODO: Add the new capacities array
n_items_new = len(values)

# 2. Create a new QuadraticProgram named "NewMDKP"
mdkp_model = QuadraticProgram(name="ConstrainedMarsMission")

# 3. Add the binary variables to the model

mdkp_model.binary_var_list(n_items_new, name='x')

# 4. Set the objective function (remember to maximize)

mdkp_model.maximize(linear=values)

# 5. Add the three linear constraints using a for loop
# TODO: What should the constraint sense be? '<=', '>=', or '=='?
#       Think: Are we setting a MAXIMUM limit or MINIMUM requirement?
for j in range(len(capacities)):
    mdkp_model.linear_constraint(
        linear=weights[j],
        sense="?",                         # TODO: Replace "?" with correct sense: '<=', '>=', or '=='
        rhs=capacities[j],
        name=f"constraint_{j}"
    )

<details>
<summary> Click here for solution </summary>

```python
import numpy as np
from qiskit_optimization import QuadraticProgram

# Problem data
values = np.array([12, 10, 15, 8, 5])
weights = np.array([
    [5, 3, 6, 3, 2],  # Mass
    [2, 4, 5, 1, 3],  # Volume
    [4, 5, 2, 6, 1]   # Power
])
capacities = np.array([12, 10, 11])
n_items = len(values)

# Create the QuadraticProgram
mdkp_model = QuadraticProgram(name="ConstrainedMarsMission")

# Create binary variables
# Note: names will be x0, x1, x2, x3, x4 (indices 0..4)
mdkp_model.binary_var_list(n_items, name='x')

# Objective: maximize sum(values[i] * x_i)
mdkp_model.maximize(linear=values)

# Physical capacity constraints
for j in range(len(capacities)):
    mdkp_model.linear_constraint(
        linear=weights[j],  # aligned with variable order x0..x4
        sense="<=",
        rhs=capacities[j],
        name=f"capacity_constraint_{j}"
    )
```
</details>

---

### 📜 Directive #1: The Power Dependency

> **“The Drone Scout (Item 1) is useless without a dedicated power source. If you select the Drone Scout, you MUST also select the Emergency Battery (Item 4).”**

This is a **conditional constraint** — think about how to express *“if item 1 is selected, then item 4 must also be selected”* using your binary variables `x_i ∈ {0,1}`.

<details>
<summary>💡 Need a hint? Click to reveal the mathematical formulation</summary>

We model this with the inequality:  
$$x_1 \leq x_4 \quad \implies \quad x_1 - x_4 \leq 0$$

✅ **Why this works**:  
- If `x₁ = 1` (Drone Scout selected), then `x₄` must be `1` to satisfy the constraint.  
- If `x₁ = 0`, `x₄` can be `0` or `1` — no restriction.  
This enforces the logical implication: `x₁ → x₄`.

</details>

---

### 📜 Directive #2: The Exclusive Equipment

> **“The Soil Analyzer (Item 0) and the BioLab Kit (Item 2) use the same mounting bracket on the lander. You can take one or the other, but NOT BOTH.”**

This is a **mutually exclusive constraint** — how can you prevent both items from being selected at the same time?

<details>
<summary>💡 Need a hint? Click to reveal the mathematical formulation</summary>

We model this with the inequality:  
$$x_0 + x_2 \leq 1$$

✅ **Why this works**:  
- If `x₀ = 1`, then `x₂` must be `0`.  
- If `x₂ = 1`, then `x₀` must be `0`.  
- If both are `0`, that’s fine too.  
Only `x₀ + x₂ = 2` is forbidden — which is exactly what we want.

</details>

In [ ]:
# --- Logical constraints (use integer indices) ---

# 📜 Power Dependency: "If you take Drone Scout (x₁), you MUST take Emergency Battery (x₄)"
# Mathematical form: 
mdkp_model.linear_constraint(
    linear={1: ???, 4: ???},  # 🤔 TODO: What coefficients enforce
    sense="???",              # 🤔 TODO: Which inequality sense? ('<=' or '>=' or '==')
    rhs=???,                  # 🤔 TODO: What should the right-hand side be?
    name="power_dependency"
)

# 📜 Exclusive Equipment: "Soil Analyzer (x₀) and BioLab Kit (x₂) — choose at most one"
# Mathematical form: 
mdkp_model.linear_constraint(
    linear={0: ???, 2: ???},  # 🤔 TODO: What coefficients enforce 
    sense="???",              # 🤔 TODO: Which inequality sense?
    rhs=???,                  # 🤔 TODO: What should the right-hand side be?
    name="exclusive_equipment"
)

print("--- Complete Problem Formulation ---")
print(mdkp_model.prettyprint())


<details>
<summary> Click here for solution </summary>

```python


# --- Logical constraints (use integer indices) ---

# Power Dependency: x_1 - x_4 <= 0   (as you specified)
mdkp_model.linear_constraint(
    linear={1: 1, 4: -1},
    sense="<=",
    rhs=0,
    name="power_dependency"
)

# Exclusive Equipment: x_0 + x_2 <= 1
mdkp_model.linear_constraint(
    linear={0: 1, 2: 1},
    sense="<=",
    rhs=1,
    name="exclusive_equipment"
)

print("--- Complete Problem Formulation ---")
print(mdkp_model.prettyprint())

```
</details>

---
## 2. Modeling and Solving the Full Problem

Now, let's build the complete model in Qiskit, including the physical capacity limits and the new logical constraints, and then solve it with QAOA.

In [ ]:
# Step 3: Configure the quantum solver and find the solution

# Configure the QAOA algorithm
sampler = Sampler()
optimizer = COBYLA()
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=3) # Using reps=3 for a better search

# Create the main solver object
qaoa_solver = MinimumEigenOptimizer(min_eigen_solver=qaoa)

# Solve the problem
result = qaoa_solver.solve(mdkp_model)

# Print and analyze the new optimal solution
print("\n--- Solution for Constrained Problem ---")
print(f"Optimal selection: {result.x}")
print(f"Maximum value: {result.fval}")

# Step 4: (Bonus) Verify the solution against ALL constraints
print("\n--- Verification of Solution ---")
solution_vector = result.x

# Check physical constraints
for j in range(len(capacities)):
    total_weight = np.dot(weights[j], solution_vector)
    is_valid = total_weight <= capacities[j]
    print(f"Capacity Constraint {j}: Total = {total_weight}, Limit = {capacities[j]}. Valid: {is_valid}")

# Check logical constraints
power_valid = solution_vector[1] <= solution_vector[4]
exclusive_valid = solution_vector[0] + solution_vector[2] <= 1
print(f"Power Dependency (x1 <= x4): Valid: {power_valid}")
print(f"Exclusive Equipment (x0 + x2 <= 1): Valid: {exclusive_valid}")

### 🤔 Analyze Your Results

1.  **Check the Selection:** Look at the items selected in `Optimal selection`. Does the new cargo list respect the two directives from Mission Control?
2.  **The Cost of Constraints:** Adding constraints restricts the available choices. This often leads to a lower optimal value, but the resulting solution is the best one that is **actually feasible** in the real world. You've successfully traded a bit of theoretical value for a mission plan that will actually work!